# Collision times for cyclists and pedestrians in key locations in Oxford 

This work is to help inform discussions about the operational times of any new bus gates installed in the city via COTS. 

It uses the following data sources:

- Road traffic collision data from https://bikedata.cyclestreets.net (which reformatted version of [UK road safety data](https://www.data.gov.uk/dataset/cb7ae6f0-4be6-4935-9277-47e5ce24a11f/road-safety-data))
- (maybe) some baseline cyclists and pedestrian numbers from data scraped from https://oxbike.co.uk/ 

## TODO

- Select locations to analyse, given potential COTS bus gate locations
- Get datasets from these locations
- Pull out time of day information
- Graph this information
- Potentially normalise with baseline data from oxbike.co.uk

## Locations to analyse

### Potential bus gate locations

- Hythe Bridge Street
- St Clements
- St Cross Street
- Hollow Way 
- Marston Ferry Road

### Roads / junctions to analyse 

- The Plain (`01_the_plain.csv`)
- High Street, Longwall junction and down to Plain (not including the Plain) (`02_high_street.csv`)
- St Clement's / Headington Road, up to Headley Way (including that junction) (`03_st_clements.csv`)
- Cowley Road, upto the Church Cowley Road (including that junction) (`04_cowley_road.csv`)
- Iffley Road, upto junction with Church Cowley Road (including that junction) (`05_iffley_road.csv`)
- Hollow Way (`06_hollow_way.csv`)
- Marston Ferry Road/Cherwell Drive (`07_marston_ferry_road.csv`)
- A4144 from station to St Giles Road (`08_a4144.csv`)

### Lower priority locations to analyse

- London Road (upto but not including the junction with Headley Way)
- Iffley Road south of Church Cowley Road (upto but not including the junction with Church Cowley Road)
- Church Cowley Road 
- Morrell Avenue/Warneford Lane/Old Road (upto the junction with the Slade)
- The Slade 
- Windmill Road 

### Getting data from https://bikedata.cyclestreets.net

There is a dragging tool for extracting data from the map. 

- Drag around each road / junction (make a judgement call RE boundaries as sometimes the lat/longs are a bit off so you have to *e.g.* go wider round the road at points)
- Select the widest possible date range (1999-2020) and export as spreadsheet. 
- Note there is a limit of 400 data points per export one has to extract in chunks by batching up by date. 
- Difficult to tease apart *e.g.* what is "Iffley Road" and what is "The Plain", made a judgement call by looking at the map. 
